In [ ]:
pip install pingouin

In [ ]:
pip install pyampute

In [ ]:
!pip uninstall -y xgboost
!pip install xgboost==2.1.2

In [ ]:
pip install shap

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
EP RQ2 Model Final

Author: Joseph Toland; Validation: Anchal Bansal
"""

# Importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import statsmodels.api as sm
import scipy.stats as stats
import io
import matplotlib.patches as mpatches

from scipy.stats import ttest_1samp, spearmanr

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (roc_auc_score, accuracy_score, confusion_matrix, roc_curve, auc)

from sklearn.inspection import permutation_importance
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor

from shapely.geometry import Point

from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap
from google.colab import files

# Setting seaborn style
sns.set(style='white')  # white, darkgrid


In [ ]:
# Upload a CSV file using files.upload()
uploaded = files.upload()

# Get the first uploaded file (assuming it's a CSV)4
uploaded_file_name = list(uploaded.keys())[0]

# Create a DataFrame from the uploaded CSV file
source = pd.read_csv(io.StringIO(uploaded[uploaded_file_name].decode('utf-8')))

#Survey Validation

In [ ]:
# Upload the CSV
uploaded = files.upload()

# Get the first uploaded file (assuming it's a CSV)4
uploaded_file_name = list(uploaded.keys())[0]

# Create a DataFrame from the uploaded CSV file
source_CN = pd.read_csv(io.StringIO(uploaded[uploaded_file_name].decode('utf-8')))

source_CN.dropna(inplace = True) #to check a clean version with all NA removed for Cronback and rWg. Use: SURVEY_ANALYSIS_FINAL_RQ2_101224_CRONBACH.csv
source_CN['P-WSAFETY-10'] = pd.to_numeric(source_CN['P-WSAFETY-10'], errors='coerce')

uploaded = files.upload()

# Get the first uploaded file (assuming it's a CSV)4
uploaded_file_name = list(uploaded.keys())[0]

# Create a DataFrame from the uploaded CSV file #to check missingness with NO NA removed. Use: SURVEY_ANALYSIS_FINAL_RQ2_101224_TESTS.csv
source_T = pd.read_csv(io.StringIO(uploaded[uploaded_file_name].decode('utf-8')))

Run Little MCAR Test  

In [ ]:
# Little MCAR TEST
import pyampute as pyampute
from pyampute.exploration.mcar_statistical_tests import MCARTest

# Conduct the MCAR test
mt = MCARTest(method="little")
p_value = mt.little_mcar_test(source_T)  # This should directly return the p-value.  Note be sure to use the data from BEFORE demogrpahic varibles have been matched to categorical reference categories.

# Print the p-value
print("P-Value from Little's MCAR Test:", p_value)

# Interpret the p-value
if p_value < 0.05:
    print("The missing data is not MCAR (reject the null hypothesis).")
else:
    print("The missing data can be considered MCAR (fail to reject the null hypothesis).")


 Cronback Alpha Test

In [ ]:
from pingouin import cronbach_alpha

WSAFE = source_CN[['P-WSAFETY-1', 'P-WSAFETY-2', 'P-WSAFETY-3', 'P-WSAFETY-4',
       'P-WSAFETY-5', 'P-WSAFETY-6', 'P-WSAFETY-7', 'P-WSAFETY-8',
       'P-WSAFETY-9', 'P-WSAFETY-10', 'P-WSAFETY-11']]

EQF = source_CN[['P-SOC-EQF-1', 'P-SOC-EQF-2', 'P-SOC-EQF-3', 'P-SOC-EQF-4']]

TRUST = source_CN[['P-TRUST-1', 'P-TRUST-2', 'P-TRUST-3', 'P-TRUST-4', 'P-TRUST-5',
       'P-TRUST-6', 'P-TRUST-7', 'P-TRUST-8', 'P-TRUST-9', 'P-TRUST-10',
       'P-TRUST-11', 'P-TRUST-12', 'P-TRUST-13', 'P-TRUST-14', 'P-TRUST-15',
       'P-TRUST-16', 'P-TRUST-17', 'P-TRUST-18', 'P-TRUST-19']]

alpha_result = cronbach_alpha(WSAFE)
print("Cronbach's Alpha for W-SAFETY:", round(alpha_result[0], 2))

alpha_result = cronbach_alpha(TRUST)
print("Cronbach's Alpha for W-TRUST:", round(alpha_result[0], 2))

alpha_result = cronbach_alpha(EQF)
print("Cronbach's Alpha for P-SOC-EQF:", round(alpha_result[0], 2))

One Sample, Two-Sided T-Test for the RwG Test

In [ ]:
# Specify the hypothesized mean
hypothesized_mean = 3

# Create a dictionary to store the results and list of DataFrames
results = {}
dataframes = [WSAFE, EQF, TRUST]

# Perform one-sample t-test for each DataFrame
for df_label, df in zip(['WSAFE', 'EQF', 'TRUST'], dataframes):
    # Create a new DataFrame for each column
    df_columns = {col: df[col] for col in df.columns}

    # Remove NA values for each column
    df_no_na = {col: col_data.dropna() for col, col_data in df_columns.items()}

    # Perform the one-sample t-test for each column again
    t_statistic = []
    p_value = []
    for col, col_data in df_no_na.items():
        t_stat, p_val = ttest_1samp(col_data, hypothesized_mean)
        t_statistic.append(t_stat)
        p_value.append(p_val)

    # Store the results in the dictionary
    results[df_label] = {'t_statistic': t_statistic, 'p_value': p_value}

# Create a DataFrame from the results dictionary
results_df = pd.DataFrame(results)

# Apply the styling to the 'p_value' column
styled_results = results_df.style.applymap(lambda x: highlight_significance(x) if 'p_value' in x else '')

# Display the styled DataFrame. This take some interpretation, we find all p values greater than 0.05 for investigation that fail the test. Then associated the predictor based on the order.  These are then selected for RwG test.
styled_results


rWG Within Group Reliability Test

In [ ]:
# Function to calculate rWG for Likert scale variables without grouping

variable_columns = ['P-WSAFETY-1', 'P-WSAFETY-4', 'P-WSAFETY-5','P-TRUST-1', 'P-TRUST-3'] #these must be from the p values that are high above where we compare them all to see if there is a central tendency selection bias

def calculate_rWG(data, variable_columns):
    # Step 1: Remove NaN values from the specified columns
    cleaned_data = data[variable_columns].dropna()

    # Step 2: Calculate Total Variance for Likert scale variables
    total_variance = np.var(cleaned_data.values.flatten(), ddof=1)

    # Step 3: Calculate Within-Group Variance for Likert scale variables
    within_group_variance = cleaned_data.apply(lambda x: ((x - x.mean())**2).sum()).mean()

    # Step 4: Calculate rWG for Likert scale variables
    k = len(variable_columns)
    rWG = within_group_variance / (total_variance + (k - 1) * within_group_variance)

    return rWG

# Calculate rWG for all variables together, removing NaN values
rWG_value = calculate_rWG(source_CN, variable_columns)

# Print the result
print(f'rWG Value for all variables: {rWG_value}')


Class imbalance test

In [ ]:
import pandas as pd

# Check for class imbalance in all binary/categorical columns
threshold_ratio = 0.1  # Threshold for imbalance (10:1 imbalance)

# Iterate through each column in the dataset
for column in source.columns:
    # Skip non-binary columns
    if source[column].nunique() > 2:
        print(f"Skipping column '{column}' (not binary/categorical).")
        continue

    print(f"\nChecking imbalance for column: {column}")

    # Get the distribution of classes
    class_distribution = source[column].value_counts()
    print("Class distribution:")
    print(class_distribution)

    # Calculate imbalance ratio
    imbalance_ratio = class_distribution.min() / class_distribution.max()

    # Print whether the column exceeds the imbalance threshold
    if imbalance_ratio < threshold_ratio:
        print(f"Imbalance ratio: {imbalance_ratio:.2f}. This exceeds the 10:1 threshold, indicating class imbalance.")
    else:
        print(f"Imbalance ratio: {imbalance_ratio:.2f}. This does not exceed the 10:1 threshold, indicating no class imbalance.")


#Descriptive Statistics, Chi-Sqares Tests, and Corelations

Descriptive Statistics

In [ ]:
# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
mean_std_min_max_summary = source.describe().loc[['mean', 'std', 'min', 'max']].round(2)
print(mean_std_min_max_summary)

Chi-square tests

In [ ]:
source_CHI = source

source_CHI.dropna(inplace=True)  # Clean version with all NAs removed (n = 181).

# Select categorical variables for Chi-square tests
chi = source_CHI[['R-WINSEC', 'P-OWNRENT', 'P-WWELL', 'P-WMUNICIPAL', 'P-WCHANGE',
                  'P-WRELIEF', 'P-WPURCHASE', 'P-RISK-W', 'P-WTREAT', 'P-ACTION-R',
                  'P-EDU-HS', 'P-INCOME-50', 'P-GENDER', 'P-WORK-Y']]

variables = chi.columns.tolist()
results = pd.DataFrame(columns=variables, index=variables)

for i, var1 in enumerate(variables):
    for j, var2 in enumerate(variables):
        if i > j:
            contingency_table = pd.crosstab(chi[var1], chi[var2])
            chi2, p, _, _ = stats.chi2_contingency(contingency_table)
            results.at[var1, var2] = round(p, 2)
        elif i == j:  # On the diagonal
            results.at[var1, var2] = np.nan

# Highlight p-values
def highlight_associated(val):
    if np.isnan(val):
        return "color: rgba(0, 128, 0, 0)"
    if val <= 0.01:
        return "color: red"
    if val <= 0.05:
        return "color: orange"
    return ""

# Style and display the DataFrame
styled_results = results.style.format(precision=2).applymap(highlight_associated)
styled_results


Spearman Rank Correlation

In [ ]:
# Selecting only the Likert and all other non-categorical variables
spear = source[['P-SOC-EQF','P-WSAFETY','P-TRUST','P-HH-POP', 'P-HH-CHILD','P-WQUANT-G', 'P-WQUANT-EM']] #note, chldren and large families are correlated, as are water stoarge and eemergneyc storage for SI

# Calculating Spearman's rank correlation coefficients
correlations = spear.corr(method='spearman')

# Calculating p-values for correlations
p_values = spear.corr(method=lambda x, y: spearmanr(x, y)[1])[['P-SOC-EQF','P-WSAFETY','P-TRUST','P-HH-POP', 'P-HH-CHILD','P-WQUANT-G', 'P-WQUANT-EM']]

pd.options.display.float_format = '{:.2f}'.format

print("Correlations:")
print(correlations[['P-SOC-EQF','P-WSAFETY','P-TRUST','P-HH-POP', 'P-HH-CHILD','P-WQUANT-G','P-WQUANT-EM']])
print("\nP-values:")
print(p_values)


Data Exploration: BoxPlots

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming 'source' is your DataFrame
source_BOXBI = source.copy()
source_BOXBI.dropna(inplace=True)  # Drop rows with missing values
source_BOXBI = source_BOXBI.drop(['INDEX'], axis=1)  # Drop the 'INDEX' column

# Mapping of variable codes to their simplified descriptions
variable_mapping = {
    'R-HWISE': 'HWISE score',
    'R-WINSEC': 'Water insecure (HWISE)',
    'P-WWELL': 'Household normally used\nprivate well water',
    'P-WMUNICIPAL': 'Household normally used\nmunicipal water',
    'P-WCHANGE': 'Change in water source',
    'P-WRELIEF': 'Used donated bottled\nwater (after)',
    'P-WPURCHASE': 'Purchased bottled\nwater (after)',
    'P-HH-POP': 'Number of people in\nhousehold',
    'P-HH-CHILD': 'Number of children\nin household',
    'P-INCOME-50': 'Income (< 50K)',
    'P-OWNRENT': 'Tenure (Owner)',
    'P-EDU-HS': 'Education (12 years or less)',
    'P-WORK-Y': 'Employment (Working full\nor part time)',
    'P-GENDER': 'Gender (Female)',
    'P-AGE-65': 'Age (> 65 years)',
    'P-WSAFETY': 'Perceptions of water safety',
    'P-TRUST': 'Perceptions of trust',
    'P-SOC-EQF': 'Perceptions of equity\nand fairness',
    'P-RISK-W': 'Told drinking water\nwas unsafe',
    'P-WTREAT': 'Installed a home water treatment\nsystem or filters',
    'P-ACTION-R': 'Relocation',
    'P-WQUANT-EM': 'Surplus/ Deficit emergency\ndrinking water',
    'P-WQUANT-G': 'Water storage (gallons)',
    'P-INT-1': 'Interaction: Water safety and\nchange',
    'P-INT-2': 'Interaction: Water safety,\nchange, and used\ndonated water'
}

# Rename the columns in the DataFrame
source_BOXBI.rename(columns=variable_mapping, inplace=True)

# Updated list of variables to plot (after renaming)
variables = list(variable_mapping.values())

# Create a figure with subplots
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(18, 18))  # 5 rows, 5 columns for 25 plots

# Flatten the axes to make it easier to loop through
axes = axes.flatten()

# Loop through each variable and create a boxplot
for i, var in enumerate(variables):
    ax = axes[i]

    # Create box plot for each variable
    source_BOXBI.boxplot(column=var, ax=ax, patch_artist=True,  # Patch for color fill
                         boxprops=dict(facecolor='lightgrey', color='black'),  # Box color
                         whiskerprops=dict(color='black'),  # Whisker color
                         capprops=dict(color='black'))  # Cap color


    # Set only the Y-axis label
    ax.set_ylabel('Values', fontsize=12)  # Y-axis label

    # Remove the x-axis label by not setting it
    ax.set_xlabel(None)

    # Optionally adjust y-axis to fit data better (if needed)
    ax.set_ylim(source_BOXBI[var].min() - 1, source_BOXBI[var].max() + 1)

# Adjust layout to prevent overlap and give space for titles
fig.tight_layout()
plt.subplots_adjust(top=0.90)  # Adjust the space at the top to avoid title overlap

# Customize font and style for publication
sns.set_style("whitegrid")  # Use a clean, professional background
plt.rcParams.update({'font.size': 10, 'font.family': 'serif'})  # Update font size and style

# Save the plot as a high-quality image (300 DPI) for publication
plt.savefig("boxplot_matrix.png", dpi=600, bbox_inches='tight')

# Show the plot
plt.show()


# Logistic Regression Model

Process data

In [ ]:
source_LR = source.drop(['INDEX'],axis=1)
source_LR.dropna(inplace = True) #to check a clean version with all drop these to get to 181:

In [ ]:
#Defining Xs, and Y
X = source_LR.drop(columns=['R-WINSEC', 'R-HWISE','P-INT-1', 'P-INT-2'])  #add or remove interaction effects
Y = source_LR[['R-WINSEC']]

columns_to_scale = ['P-WQUANT-G', 'P-WQUANT-EM', 'P-WSAFETY', 'P-TRUST','P-SOC-EQF' ] # We scale these columns so all values of coefficent are comparable by magnitude

# Initialize MinMaxScaler
scaler = MinMaxScaler()

# Scale the specified columns
X[columns_to_scale] = scaler.fit_transform(X[columns_to_scale])


Logisitc regression model evaluation

In [ ]:
def calculate_vif(X):
    """Calculate Variance Inflation Factor (VIF) for each feature."""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    return vif_data

def backward_selection(X, target, significance_level=0.1, max_iter=1000):
    """Perform backward feature selection with logistic regression."""
    remaining_features = list(X.columns)
    step = 1

    while remaining_features:
        X_subset = sm.add_constant(X[remaining_features])  # Add constant term
        logit_model = sm.Logit(target, X_subset)
        result = logit_model.fit(maxiter=max_iter, disp=0)  # Fit model with max iterations

        p_values = result.pvalues[1:]  # Exclude constant term
        max_p_value = p_values.max()
        max_p_index = p_values.idxmax()

        vif_data = calculate_vif(X[remaining_features])
        vif = vif_data.loc[vif_data['Feature'] == max_p_index, 'VIF'].values[0]

        if max_p_value > significance_level:
            X_temp = X_subset.drop(columns=[max_p_index])
            logit_model_temp = sm.Logit(target, X_temp)
            result_temp = logit_model_temp.fit(maxiter=max_iter, disp=0)
            new_AIC = result_temp.aic

            print(f"Step {step}: Removing feature: {max_p_index}, p-value: {max_p_value:.4f}, VIF: {vif:.2f}, AIC without: {new_AIC:.2f}")

            # Calculate ROC curve for the temporary model
            predicted_probs = result_temp.predict(X_temp)
            fpr, tpr, _ = roc_curve(target, predicted_probs)
            roc_auc = auc(fpr, tpr)

            remaining_features.remove(max_p_index)
        else:
            break

        step += 1

    # Final model summary and plotting
    X_final = sm.add_constant(X[remaining_features])
    final_model = sm.Logit(target, X_final)
    result = final_model.fit()

    print(result.summary())

    marginal_effects = result.get_margeff(at='overall')

    # Print out the summary of the marginal effects
    marginal_effects_summary = marginal_effects.summary()
    print("\nMarginal Effects for Remaining Variables in Final Model:")
    print(marginal_effects_summary)

    coefficients = result.params
    conf_int = result.conf_int()

    #Define names
    defined_names = [
        'Constant',
        'Tenure (Owner)',
        'Change in water\nsource',
        'Used donated bottled\n water (after)',
        'Water storage',
        'Perceptions of water\nsafety',
        'Relocation',
        'Gender (female)',
        'Age (> 65)',
    ]

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)

    # Define colors for bars
    colors = ['#B4CADE'] * len(coefficients)
    if 'P-GENDER' in coefficients.index:
        colors[coefficients.index.get_loc('P-GENDER')] = '#E8E8E8'
    if 'P-ACTION-R' in coefficients.index:
        colors[coefficients.index.get_loc('P-ACTION-R')] = '#E8E8E8'

    if len(defined_names) != len(coefficients):
        print("Warning: Length of defined names does not match length of coefficients.")

    coefficients.index = defined_names[:len(coefficients)]
    plt.barh(coefficients.index, coefficients.values, color=colors, label='Coefficient')

    # Plot confidence intervals
    plt.errorbar(coefficients.values, coefficients.index,
                 xerr=[coefficients.values - conf_int[0], conf_int[1] - coefficients.values],
                 fmt='o', color='black', label='Confidence Interval')

    plt.axvline(x=0, color='.5')
    plt.xlabel('Coefficient Value')
    plt.title('Coefficients of Logistic Regression Model')
    plt.ylabel('Features')
    plt.legend(loc="lower right")

    # Fit model with all features for ROC curve
    X_1 = sm.add_constant(X)
    logit_model = sm.Logit(target, X_1)
    result = logit_model.fit()

    predicted_probs = result.predict(X_1)
    fpr, tpr, _ = roc_curve(target, predicted_probs)
    roc_auc = auc(fpr, tpr)

    plt.subplot(1, 2, 2)
    plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")

    plt.tight_layout()
    plt.savefig('FIGURE1.png', dpi=600, bbox_inches='tight')
    plt.show()

    print("\nVIF of remaining features:")
    vif_data = calculate_vif(X[remaining_features])
    print(vif_data)

    return result, remaining_features

result, remaining_features = backward_selection(X, Y, max_iter=1000)


Run train-test validation for LR

In [ ]:
def compute_metrics(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    return accuracy, sensitivity, specificity

def evaluate_model(X_train, X_test, Y_train, Y_test, remaining_features):
    X_train_const = sm.add_constant(X_train[remaining_features])
    final_model = sm.Logit(Y_train, X_train_const)
    result = final_model.fit(disp=0)

    X_test_const = sm.add_constant(X_test[remaining_features])
    y_pred_probs = result.predict(X_test_const)
    y_pred = (y_pred_probs >= 0.5).astype(int)

    accuracy, sensitivity, specificity = compute_metrics(Y_test, y_pred)

    # Calculate ROC curve and AUC
    fpr, tpr, _ = roc_curve(Y_test, y_pred_probs)
    roc_auc = auc(fpr, tpr)

    return accuracy, sensitivity, specificity, fpr, tpr, roc_auc

def run_experiments(X, Y, remaining_features, num_experiments=100):
    random_states = np.random.randint(1, 1001, size=num_experiments)
    accuracies, sensitivities, specificities = [], [], []
    aucs = []
    all_fpr = []
    all_tpr = []

    plt.figure(figsize=(8, 8))

    for random_state in random_states:
        try:
            train_x, test_x, train_y, test_y = train_test_split(
                X, Y,
                test_size=0.5,
                random_state=random_state,
                stratify=Y
            )
            accuracy, sensitivity, specificity, fpr, tpr, roc_auc = evaluate_model(train_x, test_x, train_y, test_y, remaining_features)
            accuracies.append(accuracy)
            sensitivities.append(sensitivity)
            specificities.append(specificity)
            aucs.append(roc_auc)

            all_fpr.append(fpr)
            all_tpr.append(tpr)

        except Exception as e:
            print(f"Error with random state {random_state}: {e}")
            continue

    if len(accuracies) == 0:
        print("No successful experiments.")
        return

    # Calculate average ROC
    mean_fpr = np.linspace(0, 1, 100)
    mean_tpr = np.mean([np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)], axis=0)

    # Calculate upper and lower bounds
    lower_tpr = np.min([np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)], axis=0)
    upper_tpr = np.max([np.interp(mean_fpr, fpr, tpr) for fpr, tpr in zip(all_fpr, all_tpr)], axis=0)

    # Finalize and show ROC plot
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')  # Diagonal line
    plt.plot(mean_fpr, mean_tpr, color='darkorange', lw=2, label='Mean ROC curve (AUC = {:.2f})'.format(np.mean(aucs)))  # Average ROC curve
    plt.fill_between(mean_fpr, lower_tpr, upper_tpr, color='lightgrey', alpha=0.5, label='ROC curve area')  # Fill area between upper and lower curves
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Average Receiver Operating Characteristic (ROC) Curve')
    plt.gca().set_aspect('equal', adjustable='box')
    plt.legend(loc="lower right")
    plt.grid()
    plt.tight_layout()
    plt.savefig('FIGURE2.png', dpi=600, bbox_inches='tight')
    plt.show()

    # Print average metrics
    avg_accuracy = np.mean(accuracies)
    avg_sensitivity = np.mean(sensitivities)
    avg_specificity = np.mean(specificities)
    avg_auc = np.mean(aucs)

    std_accuracy = np.std(accuracies)
    std_sensitivity = np.std(sensitivities)
    std_specificity = np.std(specificities)
    std_auc = np.std(aucs)

    print(f"Average Accuracy: {avg_accuracy:.2f} (±{std_accuracy:.2f})")
    print(f"Average Sensitivity: {avg_sensitivity:.2f} (±{std_sensitivity:.2f})")
    print(f"Average Specificity: {avg_specificity:.2f} (±{std_specificity:.2f})")
    print(f"Average AUC: {avg_auc:.2f} (±{std_auc:.2f})")

run_experiments(X, Y, remaining_features)

SHAP for comparison table between XGBoost and logisitc regression

In [ ]:
import shap
import statsmodels.api as sm
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Define names for features (used in outputs)
defined_names = [
    'Constant',
    'Tenure (Owner)',
    'Change in water\nsource',
    'Used donated bottled\nwater (after)',
    'Water storage',
    'Perceptions of water\nsafety',
    'Relocation',
    'Gender (female)',
    'Age (> 65)',
]

def calculate_vif(X):
    """Calculate Variance Inflation Factor (VIF) for each feature."""
    vif_data = pd.DataFrame()
    vif_data["Feature"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
    return vif_data

def backward_selection(X, target, significance_level=0.1, max_iter=1000):
    """Perform backward feature selection with logistic regression."""
    remaining_features = list(X.columns)
    step = 1

    while remaining_features:
        X_subset = sm.add_constant(X[remaining_features])  # Add constant term
        logit_model = sm.Logit(target, X_subset)
        result = logit_model.fit(maxiter=max_iter, disp=0)  # Fit model with max iterations

        p_values = result.pvalues[1:]  # Exclude constant term
        max_p_value = p_values.max()
        max_p_index = p_values.idxmax()

        vif_data = calculate_vif(X[remaining_features])
        vif = vif_data.loc[vif_data['Feature'] == max_p_index, 'VIF'].values[0]

        if max_p_value > significance_level:
            X_temp = X_subset.drop(columns=[max_p_index])
            logit_model_temp = sm.Logit(target, X_temp)
            result_temp = logit_model_temp.fit(maxiter=max_iter, disp=0)
            new_AIC = result_temp.aic

            feature_index = X.columns.get_loc(max_p_index)
            feature_name = defined_names[feature_index] if feature_index < len(defined_names) else max_p_index

            print(f"Step {step}: Removing feature: {feature_name}, p-value: {max_p_value:.4f}, VIF: {vif:.2f}, AIC without: {new_AIC:.2f}")

            remaining_features.remove(max_p_index)
        else:
            break

        step += 1

    # Final model summary and plotting
    X_final = sm.add_constant(X[remaining_features])
    final_model = sm.Logit(target, X_final)
    result = final_model.fit()

    print(result.summary())

    # Use SHAP to explain the final logistic regression model
    def predict_function(X):
        """Create a callable prediction function for SHAP."""
        return result.predict(sm.add_constant(X))

    explainer = shap.Explainer(predict_function, X_final)  # Use the callable function
    shap_values = explainer(X_final)

    # Create a summary plot (beeswarm plot)
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_final, show=False)

    # Define the custom labels for the SHAP plot
    custom_labels = [
        'Constant',
        'Water storage',
        'Relocation',
        'Age (> 65)',
        'Gender (female)',
        'Used donated bottled\nwater (after)',
        'Change in water\nsource',
        'Tenure (Owner)',
        'Perceptions of water\nsafety'
    ]

    # Modify y-tick labels after the plot is created
    num_features = len(custom_labels)
    plt.yticks(ticks=range(num_features), labels=custom_labels)

    # Save the SHAP summary plot (beeswarm plot)
    plt.savefig('shap_summary_beeswarm_plot.png', dpi=600, bbox_inches='tight')
    plt.show()

    # Optionally initialize JavaScript for SHAP plots
    shap.initjs()

    return result, remaining_features

# Assuming you have already defined X and Y as your features and target
result, remaining_features = backward_selection(X, Y, max_iter=1000)


#XGBoost Model

Data processing

In [ ]:
#Final veriables slected based on research questions and hypotheses in paper, base don n = 222 full dataset in source
X = source.drop(columns=['INDEX','R-WINSEC', 'R-HWISE','P-INT-1', 'P-INT-2'])
Y = source[['R-WINSEC']]

train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size = 0.1,random_state= 15,  stratify=source['R-WINSEC'] )

XGBoost Model Evaluation

In [ ]:
# Define and initialize the XGBoost model, based on hyperparameter selection with some final tuning
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=2,
    objective='binary:logistic',
    subsample=1,
    gamma=0,
    reg_lambda=1,
    colsample_bytree=0.8,
    colsample_bylevel=0.8,
)

# Perform 10-fold cross-validation to evaluate the model
xgb_cv_scores = cross_val_score(xgb_model, train_x, train_y, cv=10, scoring='roc_auc')
print("XGBoost Train AUC (CV): {0:.2f} (+/- {1:.2f})".format(xgb_cv_scores.mean(), xgb_cv_scores.std() * 2))

# Fit the model on the training data
xgb_model.fit(train_x, train_y)

# Make predictions and calculate metrics
xgb_train_predictions = xgb_model.predict(train_x)
xgb_test_prob_predictions = xgb_model.predict_proba(test_x)[:, 1]
xgb_full_model_predictions = xgb_model.predict_proba(X)[:, 1]
xgb_full_model_binary_predictions = (xgb_full_model_predictions > 0.5).astype(int)  # Convert probabilities to binary predictions

train_accuracy = accuracy_score(train_y, xgb_train_predictions)
test_auc = roc_auc_score(test_y, xgb_test_prob_predictions)
full_model_auc = roc_auc_score(Y, xgb_full_model_predictions)
full_model_accuracy = accuracy_score(Y, xgb_full_model_binary_predictions)
test_accuracy = accuracy_score(test_y, xgb_model.predict(test_x))

# Calculate sensitivity and specificity for the full model
conf_matrix = confusion_matrix(Y, xgb_full_model_binary_predictions)
tn, fp, fn, tp = conf_matrix.ravel()
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

# Print results
print("XGBoost Training Accuracy: {0:.2f}".format(train_accuracy))
print("XGBoost Test AUC: {0:.2f}".format(test_auc))
print("XGBoost Test Accuracy: {0:.2f}".format(test_accuracy))
print("XGBoost Full Model AUC: {0:.2f}".format(full_model_auc))
print("XGBoost Full Model Accuracy: {0:.2f}".format(full_model_accuracy))
print("XGBoost Full Model Sensitivity: {0:.2f}".format(sensitivity))
print("XGBoost Full Model Specificity: {0:.2f}".format(specificity))

XGBoost Visualization - ROC Curve

In [ ]:
# Make probability predictions on the full dataset
xgb_probs_full = xgb_model.predict_proba(X)[:, 1]

# Calculate the false positive rate, true positive rate, and thresholds
fpr_full, tpr_full, thresholds_full = roc_curve(Y, xgb_probs_full)

# Calculate the AUC for the full dataset
roc_auc_full = auc(fpr_full, tpr_full)

# Plot the ROC curve for the full dataset
plt.figure()
plt.plot(fpr_full, tpr_full, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_full)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) - Full Dataset')
plt.legend(loc="lower right")
plt.savefig('ROC_XGBt.png', dpi=600, bbox_inches='tight')
plt.show()

# Print the AUC for the full dataset
print("XGBoost Full Dataset AUC: {0:.2f}".format(roc_auc_full))

SHAP Feature Importance Summary Plot

In [ ]:
import shap

# Create a SHAP explainer object
explainer = shap.Explainer(xgb_model)

# Calculate SHAP values for all features on the full dataset X
shap_values = explainer.shap_values(X)

# Create a summary plot
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values, X, show=False)

# Define specific names for each feature
custom_labels = [
  'Private well',
	'Municipal water',
	'Education\n(high school or less)',
	'Home water treatment',
	'Gender (female)',
	'Children',
	'Income (< 50K)',
	'Perceptions of\nequity',
	'Told water unsafe',
	'Relocation',
	'Used donated bottled\nwater (after)',
	'Perceptions of trust',
  'Emergency water',
	'Change in water\nsource',
	'Household size',
	'Age (> 65)',
	'Water storage',
	'Purchased bottled\nwater (after)',
	'Tenure (Owner)',
	'Perceptions of water\nsafety'
]

# Modify y-tick labels after the plot is created
num_features = len(custom_labels)
plt.yticks(ticks=range(num_features), labels=custom_labels)

# Save the SHAP summary plot (beeswarm plot)
plt.savefig('shap_summary_beeswarm_plot.png', dpi=600, bbox_inches='tight')
plt.show()

# Optionally initialize JavaScript
shap.initjs()


Permuation Importance Test

In [ ]:
perm_importance = permutation_importance(xgb_model, X, Y, n_repeats=100, random_state=15) # run n iterations for permutation importance

# Feature names from the dataset
default_labels = X.columns.tolist()

# Custom labels for specific features
custom_labels = {
    'P-WORK-Y': 'Employment',
    'P-WMUNICIPAL': 'Municipal water',
    'P-WWELL': 'Private well',
    'P-EDU-HS': 'Education\n(high school or less)',
    'P-GENDER': 'Gender (female)',
    'P-WTREAT': 'Home water treatment',
    'P-INCOME-50': 'Income (< 50K)',
    'P-HH-CHILD': 'Children',
    'P-RISK-W': 'Told water unsafe',
    'P-AGE-65': 'Age (> 65)',
    'P-TRUST': 'Perceptions of trust',
    'P-SOC-EQF': 'Perceptions of equity',
    'P-ACTION-R': 'Relocation',
    'P-WCHANGE': 'Change in water source',
    'P-WRELIEF': 'Used donated bottled\nwater (after)',
    'P-WQUANT-EM': 'Emergency water',
    'P-WPURCHASE': 'Purchased bottled\nwater (after)',
    'P-HH-POP': 'Household size',
    'P-WQUANT-G': 'Water storage',
    'P-OWNRENT': 'Tenure (Owner)',
    'P-WSAFETY': 'Perceptions of water\nsafety'
}

# Creating labels for the plot using the custom mapping
plot_labels = [custom_labels.get(col, col) for col in default_labels]

# Plotting the permutation importance
sorted_idx = perm_importance.importances_mean.argsort()
plt.figure(figsize=(9, 9))
bars = plt.barh(range(len(sorted_idx)), perm_importance.importances_mean[sorted_idx],
                 xerr=perm_importance.importances_std[sorted_idx], align='center', color='#B5CBDF')

# Adding custom legend
plt.yticks(range(len(sorted_idx)), np.array(plot_labels)[sorted_idx])  # Use mapped labels
plt.xlabel("Permutation Importance")
plt.title("Permutation Importance of Features")

mean_importance_patch = mpatches.Rectangle((0, 0), width=0.2, height=0.1, color='#B5CBDF', edgecolor='black', label='Global importance (mean)')
std_dev_patch = plt.Line2D([0, 1], [0, 0], color='black', linestyle='-', lw=1, label='± Std. Dev.')  # Dashed line for Std Dev

# Add both patches to the legend
plt.legend(handles=[mean_importance_patch, std_dev_patch], loc="lower right")

plt.savefig('PERM_XGB.png', dpi=600, bbox_inches='tight')
plt.tight_layout()
plt.show()

Partial dependnace plots

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shap
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import LinearSegmentedColormap

# Create a custom colormap from blue (#0000F5) to red (#EA3323)
colors = ['#0000F5', '#EA3323']
cmap = LinearSegmentedColormap.from_list('custom_cmap', colors)

# Create a SHAP explainer object and calculate SHAP values
explainer = shap.Explainer(xgb_model)
shap_values = explainer.shap_values(X)

# Choose the feature of interest
feature = 'P-WSAFETY'
interaction_feature = 'P-WCHANGE'

# Get the values of the feature and the interaction feature
feature_values = X[feature]
interaction_values = X[interaction_feature]

# Compute the SHAP values
shap_feature_values = shap_values[:, X.columns.get_loc(feature)]

# Sort the data
sorted_idx = np.argsort(feature_values)
sorted_feature_values = feature_values.iloc[sorted_idx]
sorted_shap_values = shap_feature_values[sorted_idx]

# Create a figure with two subplots (one for SHAP Partial Dependence plot, and the other for the stacked bar chart)
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# --- SHAP Partial Dependence Plot ---
scatter = axs[0].scatter(
    sorted_feature_values, sorted_shap_values,
    c=interaction_values.iloc[sorted_idx], cmap=cmap, alpha=1
)

# Set plot labels, title, and axis ticks for the SHAP plot
axs[0].set_xlabel('Perceptions of Water Safety (High-Low)', fontsize=14, labelpad=15)
axs[0].set_ylabel('SHAP Value (Impact on Model)', fontsize=14, labelpad=15)
axs[0].set_title(f'Partial Dependence Plot of Change in Water Source\n (Interaction with Perception of Water Safety)', fontsize=16, pad=20)
axs[0].tick_params(axis='both', labelsize=12)
axs[0].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[0].yaxis.set_major_locator(MaxNLocator(integer=True))

# Add colorbar
cbar = plt.colorbar(scatter, ax=axs[0])
cbar.set_label('Change in Water Source', fontsize=12)
cbar.set_ticks(np.arange(int(np.min(interaction_values)), int(np.max(interaction_values)) + 1, 1))

# --- Stacked Bar Chart with Percentage Labels ---
# Create a contingency table (counts) of the two features of interest
contingency_table = pd.crosstab(X['P-WSAFETY'], X['P-WCHANGE'])

# Calculate the percentage contribution for each group
percentages = contingency_table.div(contingency_table.sum(axis=1), axis=0) * 100

# Plot the stacked bar chart on the second subplot using the custom colormap
contingency_table.plot(kind='bar', stacked=True, colormap=cmap, ax=axs[1])

# Customize the chart with labels and title
axs[1].set_xlabel('Perceptions of Water Safety (High-Low)', fontsize=14)
axs[1].set_ylabel('Count', fontsize=14)
axs[1].set_title('Partial Dependence Contribution', fontsize=16)
axs[1].tick_params(axis='both', labelsize=12)
axs[1].xaxis.set_major_locator(MaxNLocator(integer=True))
axs[1].yaxis.set_major_locator(MaxNLocator(integer=True))

# Add percentage labels to the red parts (last column of contingency table)
for i, (group, row) in enumerate(contingency_table.iterrows()):
    # Get the cumulative height of all previous bars (for positioning)
    cumulative_height = 0
    for j, value in enumerate(row):
        if j == len(row) - 1:  # If it's the last column (red part)
            percentage = percentages.iloc[i, j]
            axs[1].text(
                i, cumulative_height + value / 2,  # Position: center of the red bar
                f'{percentage:.0f}%',  # Label text
                ha='center', va='center', color='black', fontsize=10, alpha = 0
            )
        cumulative_height += value

handles, labels = axs[1].get_legend_handles_labels()

# Manually update the legend labels, e.g., '0' -> 'No', '1' -> 'Yes'
updated_labels = ['No' if label == '0' else 'Yes' for label in labels]

# Set the new legend with the updated labels
axs[1].legend(handles, updated_labels, title='Change in Water Source', fontsize=12)

plt.tight_layout()
plt.savefig('INTS.png', dpi=600, bbox_inches='tight')

# Show the combined plots
plt.show()
